# Network of Cross-overs between Fandoms

In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import scipy
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, from_networkx, output_file, save
from bokeh.models import HoverTool, ColorBar, LinearColorMapper
from bokeh.transform import linear_cmap
from bokeh.layouts import column
import matplotlib.colors as mcolors
from bokeh.embed import file_html
from bokeh.resources import CDN

## Setting up

In [3]:
tags = pd.read_csv("tags-20210226.csv")
works = pd.read_csv("works-20210226.csv")

In [4]:
# process tags
works['tags'] = works['tags'].astype(str).apply(lambda tags: tags.split('+'))

fandoms = tags[tags['type'] == "Fandom"]

In [5]:
# Filter to only columns that are needed - merge will need a lot of compute
works_small = works[["tags", "word_count"]]
fandoms_small = fandoms[['id', 'name', 'cached_count']]
works_small

,tags,word_count
0,"[10, 414093, 1001939, 4577144, 1499536, 110, 4...",388.0
1,"[10, 20350917, 34816907, 23666027, 23269305, 2...",1638.0
2,"[10, 10613413, 9780526, 3763877, 3741104, 7657...",1502.0
3,"[10, 15322, 54862755, 20595867, 32994286, 663,...",100.0
4,"[11, 721553, 54604, 1439500, 3938423, 53483274...",994.0
...,...,...
7269688,"[78, 77, 84, 101, 104, 105, 106, 23, 13, 16, 7...",705.0
7269689,"[78, 77, 84, 107, 23, 10, 16, 70, 933, 616]",1392.0
7269690,"[77, 78, 69, 108, 109, 62, 110, 23, 9, 111, 16...",1755.0
7269691,"[112, 113, 13, 114, 16, 115, 101, 117, 118, 11...",1338.0


In [6]:
works_small['tags'][0]

['10', '414093', '1001939', '4577144', '1499536', '110', '4682892', '21', '16']

In [7]:
fandoms_small

,id,name,cached_count
25,27,Supernatural,310300
27,31,Redacted,5
33,37,Boondock Saints (1999),47
38,46,Lord of the Rings RPF,3538
44,56,Gravitation (Anime),6
...,...,...,...
14466583,55393953,Карнавальная ночь | Carnival Night (1956),0
14466639,55394121,Brain Dump (Web Series),0
14466697,55394295,Redacted,3
14466959,55395081,In the Reign of Terror - G. A. Henty,0


## Cleaning & reorg

### 1. check if the numbers in each list of each row are in the "id" column of another df "fandoms_small"

In [8]:
# Convert strings to integers in the "tags" column, handling NaN values
works_small['tags'] = works_small['tags'].apply(
    lambda x: [int(tag) for tag in x if tag != 'nan']
)

works_small['tags'][0]

/var/folders/lk/jwtj64_565zdwjrcvb73p4dc0000gn/T/ipykernel_77812/239333842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  works_small['tags'] = works_small['tags'].apply(


[10, 414093, 1001939, 4577144, 1499536, 110, 4682892, 21, 16]

In [ ]:
# works_small.to_csv('works_small.csv', index=False)

In [9]:
# Convert the "id" column in fandoms_small to a set for faster lookup
fandom_ids = set(fandoms_small['id'].astype(int))

# Filter tags in works_small to only include those present in fandoms_small
works_small['filtered_tags'] = works_small['tags'].apply(lambda tags: [tag for tag in tags if tag in fandom_ids])


/var/folders/lk/jwtj64_565zdwjrcvb73p4dc0000gn/T/ipykernel_77812/1935684754.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  works_small['filtered_tags'] = works_small['tags'].apply(lambda tags: [tag for tag in tags if tag in fandom_ids])


### make edgelist
2. of the numbers in that column, count how many times a pair of them appear, and return a new df with integer 1, integer 2, and the count of each pair's appearance

In [10]:
from itertools import combinations
from collections import Counter

# Generate all possible unique pairs from each list in the filtered_tags column
pairs = []

for tags in works_small['filtered_tags']:
    # Generate combinations (unique pairs) for each list
    if len(tags) > 1:
        pairs.extend(combinations(sorted(tags), 2))

# Count the occurrences of each pair
pair_counts = Counter(pairs)


In [11]:
# Convert the pair counts to a DataFrame
pair_counts_df = pd.DataFrame(
    pair_counts.items(), columns=['ids', 'count']
)

pair_counts_df[['integer_1', 'integer_2']] = pd.DataFrame(pair_counts_df['ids'].tolist(), index=pair_counts_df.index) 
pair_counts_df = pair_counts_df.drop(columns='ids')

pair_counts_df.head()

,count,integer_1,integer_2
0,83981,414093,1001939
1,128,3741104,3763877
2,249,3741104,9780526
3,109,3741104,10613413
4,177,3763877,9780526


### Get edgelist with fandom names

In [107]:
# Merge to get the first name
merged_df_1 = pair_counts_df.merge(fandoms_small[['id', 'name']], how='left', left_on='integer_1', right_on='id')

# Rename the columns for clarity
merged_df_1 = merged_df_1.rename(columns={'name': 'name_1'})
merged_df_1 = merged_df_1.drop(columns=['id'])

merged_df_1.head()


,count,integer_1,integer_2,name_1,cached_count_1
0,83981,414093,1001939,Marvel Cinematic Universe,240536
1,128,3741104,3763877,Boku no Hero Academia,2472
2,249,3741104,9780526,Boku no Hero Academia,2472
3,109,3741104,10613413,Boku no Hero Academia,2472
4,177,3763877,9780526,BnHA,33805


In [109]:
# Merge to get the first name
merged_df_2 = merged_df_1.merge(fandoms_small[['id', 'name']], how='left', left_on='integer_2', right_on='id')

# Rename the columns for clarity
merged_df_2 = merged_df_2.rename(columns={'name': 'name_2'})
merged_df_2 = merged_df_2.drop(columns=['id'])

merged_df_2.head()


,count,integer_1,integer_2,name_1,cached_count_1,name_2,cached_count_2
0,83981,414093,1001939,Marvel Cinematic Universe,240536,The Avengers (Marvel Movies),157813
1,128,3741104,3763877,Boku no Hero Academia,2472,BnHA,33805
2,249,3741104,9780526,Boku no Hero Academia,2472,My Hero Academia,5078
3,109,3741104,10613413,Boku no Hero Academia,2472,mha,7393
4,177,3763877,9780526,BnHA,33805,My Hero Academia,5078


In [110]:
edgelist_df = merged_df_2

In [111]:
# Select and order the relevant columns
# edgelist_df = merged_df_2[['name_1', 'name_2', 'count']]

edgelist_df.head(20)

,count,integer_1,integer_2,name_1,cached_count_1,name_2,cached_count_2
0,83981,414093,1001939,Marvel Cinematic Universe,240536,The Avengers (Marvel Movies),157813
1,128,3741104,3763877,Boku no Hero Academia,2472,BnHA,33805
2,249,3741104,9780526,Boku no Hero Academia,2472,My Hero Academia,5078
3,109,3741104,10613413,Boku no Hero Academia,2472,mha,7393
4,177,3763877,9780526,BnHA,33805,My Hero Academia,5078
5,280,3763877,10613413,BnHA,33805,mha,7393
6,138,9780526,10613413,My Hero Academia,5078,mha,7393
7,17742,11987966,33035890,魔道祖师 - 墨香铜臭 | Módào Zǔshī - Mòxiāng Tóngxiù,47441,陈情令 | The Untamed (TV),30914
8,4,34431205,34782485,Redacted,5,Sands of Arawiya - Hafsah Faizal,4
9,4084,1633246,14988696,Dangan Ronpa - All Media Types,21958,Dangan Ronpa: Trigger Happy Havoc,6002


### Clean up edgelist

In [113]:
len(edgelist_df)

1054762

In [114]:
# remove redacted
edgelist_df = edgelist_df.drop(edgelist_df[(edgelist_df.name_1 == "Redacted") | (edgelist_df.name_2 == "Redacted")].index)
len(edgelist_df)

757734

In [115]:
# remove edges between fandoms and "anime - fandom" and other possibilities
edgelist_df = edgelist_df.drop(edgelist_df[(edgelist_df.name_1 == "anime - Fandom") | (edgelist_df.name_2 == "anime - Fandom")].index)
edgelist_df = edgelist_df.drop(edgelist_df[(edgelist_df.name_1 == "animation - Fandom") | (edgelist_df.name_2 == "animation - Fandom")].index)
len(edgelist_df)

757239

In [116]:
# drop all combinations of bnha, boku no hero academia, mha, my hero academia etc
edgelist_df = edgelist_df.drop(index=[1, 2, 3, 4, 5, 6])
len(edgelist_df)

# probably more cases like this - mha, bnha etc, but none with high counts as far as i can tell. also some are valid, like "Star Wars - All Media Types" connected to the Clone Wars etc

757233

In [197]:
# drop "& Related Fandoms"
# Create a boolean mask to identify rows that contain "& Related Fandoms" in either 'name_1' or 'name_2'
mask_related_fandoms = edgelist_df['name_1'].str.contains("& Related Fandoms") | edgelist_df['name_2'].str.contains("& Related Fandoms")

# Use the mask to filter out the rows
edgelist_df = edgelist_df[~mask_related_fandoms]

len(edgelist_df)

741606

In [198]:
# check
edgelist_df.head(20)

,count,integer_1,integer_2,name_1,cached_count_1,name_2,cached_count_2
0,83981,414093,1001939,Marvel Cinematic Universe,240536,The Avengers (Marvel Movies),157813
7,17742,11987966,33035890,魔道祖师 - 墨香铜臭 | Módào Zǔshī - Mòxiāng Tóngxiù,47441,陈情令 | The Untamed (TV),30914
9,4084,1633246,14988696,Dangan Ronpa - All Media Types,21958,Dangan Ronpa: Trigger Happy Havoc,6002
10,16448,218280,254648,Video Blogging RPF,39078,Minecraft (Video Game),31185
11,140,51823,251062,Forgotten Realms,675,Dungeons & Dragons (Roleplaying Game),4841
12,4437,1464,1801,Star Trek: The Original Series,15212,Star Trek,36097
13,3071,1464,601802,Star Trek: The Original Series,15212,Star Trek: Alternate Original Series (Movies),33604
14,6753,1801,601802,Star Trek,36097,Star Trek: Alternate Original Series (Movies),33604
15,242,3658,1281791,Tokyo Babylon,641,X -エックス- | X/1999,724
16,83,34606586,34674836,ケンガンアシュラ | Kengan Ashura (Manga),156,ケンガンアシュラ | Kengan Ashura (Anime),103


In [38]:
# edgelist_df.to_csv('fandom_network_edgelist.csv', index=False)

## NetworkX

In [199]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

# np.random.seed(seed=42)

output_notebook()

Loading BokehJS ...

In [200]:
# create dict mapping of ids to fandom names to secure node attributes
id_fandom_mapping = dict(fandoms_small[['id', 'name']].values)

In [201]:
# create dict mapping of id to fandom cached_count for nodes
id_cached_count_mapping = dict(fandoms_small[['id', 'cached_count']].values)
# id_cached_count

### Filter dataset

In [202]:
# cant run below code, takes too long -- need to find way to reduce dataset. maybe 1 graph w random sample, another with just the top 100?

# sns.histplot(data=edgelist_df, x="count", kde=True)

1. Random sample

In [203]:
sample_size = 50
min_count = 100

sample_edgelist = edgelist_df[edgelist_df['count'] > min_count].sample(sample_size)

2. Take top 50

In [204]:
top_num = 50

top_edgelist = edgelist_df.sort_values(by="count", ascending=False).head(top_num)

### Graph sample edgelist

In [205]:
sample_edgelist.head()

,count,integer_1,integer_2,name_1,cached_count_1,name_2,cached_count_2
34824,304,452874,8292487,Pocket Monsters | Pokemon (Main Video Game Ser...,9221,Pocket Monsters: HeartGold & SoulSilver | Poke...,508
114528,364,207864,3034556,Carmilla - J. Sheridan Le Fanu,540,Carmilla (Web Series),6611
12436,205,613839,1098294,Batgirl (Comics),1571,Red Hood and the Outlaws (Comics),5854
117344,101,541,11824,Power Rangers,3383,Power Rangers Wild Force,179
7029,123,747342,3290804,Star Wars: The Clone Wars (2008) - All Media T...,21336,Star Wars Legends: Jedi Apprentice Series - Ju...,698


In [206]:
# needs to be indices so that bokeh can work

G_sample = nx.from_pandas_edgelist(sample_edgelist, source='integer_1', target='integer_2', edge_attr='count')
G_sample.nodes(data=True)

NodeDataView({452874: {}, 8292487: {}, 207864: {}, 3034556: {}, 613839: {}, 1098294: {}, 541: {}, 11824: {}, 747342: {}, 3290804: {}, 9754024: {}, 32833447: {}, 101375: {}, 14444018: {}, 4997269: {}, 12307111: {}, 53131: {}, 77470: {}, 9892897: {}, 11413120: {}, 2850393: {}, 5171863: {}, 7266: {}, 1967132: {}, 431213: {}, 11264: {}, 96137: {}, 578887: {}, 722568: {}, 722570: {}, 4896523: {}, 643142: {}, 684029: {}, 25172: {}, 686974: {}, 449388: {}, 226657: {}, 604153: {}, 7913: {}, 14553: {}, 796047: {}, 5946839: {}, 662604: {}, 23271744: {}, 727114: {}, 1001939: {}, 253298: {}, 414093: {}, 741433: {}, 82576: {}, 42533587: {}, 27: {}, 813205: {}, 586439: {}, 10801369: {}, 110511: {}, 254911: {}, 254632: {}, 46023568: {}, 5316694: {}, 11987966: {}, 26574617: {}, 449385: {}, 3491546: {}, 2198962: {}, 2401815: {}, 69383: {}, 1002903: {}, 6943: {}, 19428: {}, 606566: {}, 109430: {}, 969647: {}, 841899: {}, 978202: {}, 379999: {}, 1116036: {}, 193591: {}, 130638: {}, 127710: {}, 223664: {}

**Node attributes**

In [207]:
# set node attribudes for fandom names
nx.set_node_attributes(G_sample, name='fandom', values=id_fandom_mapping)

# i want to color by cached_count of each fandom
nx.set_node_attributes(G_sample, name='cached_count', values=id_cached_count_mapping)

# need to adjust cached_count because too big distance between numbers
adjusted_cached_count_mapping = {k: np.log1p(v) for k, v in id_cached_count_mapping.items()}
nx.set_node_attributes(G_sample, name='adjusted_cached_count', values=adjusted_cached_count_mapping)

In [208]:
# node attributes for degree 
degrees = dict(nx.degree(G_sample))
nx.set_node_attributes(G_sample, name='degree', values=degrees)

In [209]:
# Slightly adjust degree so that the nodes with very small degrees are still visible
number_to_adjust_by = 10
adjusted_node_size = dict([(node, degree/number_to_adjust_by) for node, degree in nx.degree(G_sample)])
nx.set_node_attributes(G_sample, name='adjusted_node_size', values=adjusted_node_size)

In [210]:
#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_cached_count' #'#860101' 

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Reds8

#Choose a title!
title_sample = 'Fandom Network by Cross-overs, sample of ' + str(sample_size) + ' more than ' + str(min_count) + ' fics'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
        ("Fandom", "@fandom"),
        ("Degree", "@degree"),
        ("Count", "@cached_count")
]

#Create a plot — set dimensions, toolbar, and title
plot_sample = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-1.5, 1.5), y_range=Range1d(-1.5, 1.5), title=title_sample)

# pos = nx.spring_layout(G_sample, k=0.2, iterations=1000)

pos_sample = nx.fruchterman_reingold_layout(G_sample, k=0.2, iterations=1000)

graph_sample = from_networkx(G_sample, pos_sample, scale=1, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color_sample = min(graph_sample.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color_sample = max(graph_sample.node_renderer.data_source.data[color_by_this_attribute])
graph_sample.node_renderer.glyph = Circle(radius=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, maximum_value_color_sample, minimum_value_color_sample), fill_alpha=0.7)

# graph_sample.node_renderer.glyph = Circle(radius=size_by_this_attribute, fill_color=color_by_this_attribute, fill_alpha=0.5)
# graph_sample.edge_renderer.glyph = MultiLine(line_alpha=1, line_width=4, line_color="black")

plot_sample.renderers.append(graph_sample)

show(plot_sample)



Interpretation: the bigger the node, the greater the degree. the darker the color, the more fics in the fandom.

### Graph top 100

In [211]:
top_edgelist.head()

,count,integer_1,integer_2,name_1,cached_count_1,name_2,cached_count_2
0,83981,414093,1001939,Marvel Cinematic Universe,240536,The Avengers (Marvel Movies),157813
46,51455,414093,586439,Marvel Cinematic Universe,240536,Captain America (Movies),95654
47,35409,586439,1001939,Captain America (Movies),95654,The Avengers (Marvel Movies),157813
156,31923,116304,242462,A Song of Ice and Fire - George R. R. Martin,47683,Game of Thrones (TV),54688
366,27523,727114,1001939,The Avengers (Marvel) - All Media Types,66345,The Avengers (Marvel Movies),157813


Noticing that a lot of the above 'crossovers' are just within the same fandom e.g. The Avengers (Marvel) - All Media Types and The Avengers (Marvel Movies). A lot of these are authors just tagging multiple times for the same fandom, which isn't the same as a cross-over. But! This doesn't mean that 2 tags with "All Media Types" could come together to be a crossover from 2 different fandoms. It's just more common for the bigger fandoms to have the possibility of tagging "All Media Types".

Going to remove "All Media Types" for this edgelist of top 50 crossovers, so there's more information. Going to leave the random sample as it is, because it doesn't have this high incidence of "All Media Types" invalidating 'crossovers'.

In [227]:
len(edgelist_df)

741606

In [228]:
# Create a boolean mask to identify rows that contain "& Related Fandoms" in either 'name_1' or 'name_2'
mask_all_media_types = edgelist_df['name_1'].str.contains("All Media Types") | edgelist_df['name_2'].str.contains("All Media Types")

# Use the mask to filter out the rows
top_edgelist_df = edgelist_df[~mask_all_media_types]
len(top_edgelist_df)

666344

In [229]:
top_edgelist = top_edgelist_df.sort_values(by="count", ascending=False).head(top_num)

In [230]:
# needs to be indices so that bokeh can work

G_top = nx.from_pandas_edgelist(top_edgelist, source='integer_1', target='integer_2', edge_attr='count')
G_top.nodes(data=True)

NodeDataView({414093: {}, 1001939: {}, 586439: {}, 116304: {}, 242462: {}, 226657: {}, 7266: {}, 114591: {}, 27251507: {}, 245368: {}, 858574: {}, 1080663: {}, 406: {}, 27785: {}, 11987966: {}, 33035890: {}, 218280: {}, 254648: {}, 32833447: {}, 115613: {}, 224545: {}, 10801369: {}, 229522: {}, 134900: {}, 34555820: {}, 114961: {}, 873394: {}, 82576: {}, 969647: {}, 1801: {}, 601802: {}, 28642: {}, 390: {}, 879346: {}, 235690: {}, 23985107: {}, 21872622: {}, 34135873: {}, 25760640: {}, 169: {}, 13253: {}, 96137: {}, 10223: {}, 22034: {}, 81269: {}, 4231202: {}, 289604: {}, 870188: {}, 5437936: {}, 1464: {}, 1072216: {}, 6856609: {}, 15270224: {}, 587792: {}, 2282814: {}, 5316694: {}, 2428: {}})

In [231]:
# set node attribudes for fandom names
nx.set_node_attributes(G_top, name='fandom', values=id_fandom_mapping)

# i want to color by cached_count of each fandom
nx.set_node_attributes(G_top, name='cached_count', values=id_cached_count_mapping)

# need to adjust cached_count because too big distance between numbers
nx.set_node_attributes(G_top, name='adjusted_cached_count', values=adjusted_cached_count_mapping)

In [232]:
# node attributes for degree 
degrees = dict(nx.degree(G_top))
nx.set_node_attributes(G_top, name='degree', values=degrees)

In [248]:
# Slightly adjust degree so that the nodes with very small degrees are still visible
number_to_adjust_by = 10
adjusted_node_size = dict([(node, degree/number_to_adjust_by) for node, degree in nx.degree(G_top)])
nx.set_node_attributes(G_top, name='adjusted_node_size', values=adjusted_node_size)

In [249]:
#Choose a title!
title = 'Fandom Network by Cross-overs, ' + str(top_num) + ' most cross-overs'

#Create a plot — set dimensions, toolbar, and title
plot_top = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-1.5, 1.5), y_range=Range1d(-1.5, 1.5), title=title)

# pos_top = nx.spring_layout(G_top, k=0.2, iterations=1000)

pos_top = nx.fruchterman_reingold_layout(G_top, k=0.3, iterations=1000)

graph_top = from_networkx(G_top, pos_top, scale=1, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color_top = min(graph_top.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color_top = max(graph_top.node_renderer.data_source.data[color_by_this_attribute])
graph_top.node_renderer.glyph = Circle(radius=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, maximum_value_color_top, minimum_value_color_top), fill_alpha=0.7)

plot_top.renderers.append(graph_top)

show(plot_top)


In [237]:
top_edgelist

,count,integer_1,integer_2,name_1,cached_count_1,name_2,cached_count_2
0,83981,414093,1001939,Marvel Cinematic Universe,240536,The Avengers (Marvel Movies),157813
46,51455,414093,586439,Marvel Cinematic Universe,240536,Captain America (Movies),95654
47,35409,586439,1001939,Captain America (Movies),95654,The Avengers (Marvel Movies),157813
156,31923,116304,242462,A Song of Ice and Fire - George R. R. Martin,47683,Game of Thrones (TV),54688
471,27519,226657,414093,Iron Man (Movies),46331,Marvel Cinematic Universe,240536
43,25135,7266,414093,Marvel,147757,Marvel Cinematic Universe,240536
437,24336,114591,27251507,Good Omens - Neil Gaiman & Terry Pratchett,39153,Good Omens (TV),36865
1415,23668,226657,1001939,Iron Man (Movies),46331,The Avengers (Marvel Movies),157813
390,21506,245368,414093,Thor (Movies),45033,Marvel Cinematic Universe,240536
1132,19015,858574,1080663,Dragon Age (Video Games),34111,Dragon Age: Inquisition,39409
